In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from datetime import date, datetime

# Abrindo arquivo de engajamento da tela principal

In [2]:
engajamento = pd.read_csv(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Engajamento_tela1.csv", sep=';', encoding='utf-8')
print(engajamento)

           Data  Segundos
0    03/08/2023        24
1    04/08/2023         6
2    05/08/2023         5
3    06/08/2023         6
4    07/08/2023         8
..          ...       ...
251  10/04/2024        27
252  11/04/2024        14
253  12/04/2024        29
254  13/04/2024        17
255  14/04/2024        17

[256 rows x 2 columns]


# Realizando o corte do arquivo por data

In [3]:
data_inicio = '02-08-2023'
data_inicio_titulo = data_inicio
data_inicio = datetime.strptime(data_inicio, '%d-%m-%Y').date()
data_fim = '16-04-2024'
data_fim_titulo = data_fim
data_fim = datetime.strptime(data_fim, '%d-%m-%Y').date()
engajamento['data_corte'] = ''
for ind in range(0, len(engajamento)):
    engajamento['data_corte'][ind] = datetime.strptime(engajamento['Data'][ind], '%d/%m/%Y').date()
engajamento = engajamento[engajamento['data_corte'] >= data_inicio]
engajamento = engajamento[engajamento['data_corte'] <= data_fim]

# Gráfico de Engajamento médio diário da tela principal

In [12]:
fig = px.line(engajamento, x="Data", y="Segundos", title= 'Tela Principal engajamento médio de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Engajamento_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

## Preparação do arquivo para relatório mensal

In [5]:
engajamento['mes'] = 0
engajamento['ano'] = 0
engajamento['mes_ano'] = 0
for ind in range(0, len(engajamento)):
    data_wrk = datetime.strptime(engajamento['Data'][ind], '%d/%m/%Y').date()
    engajamento['mes_ano'][ind] = str(data_wrk.year) + str(data_wrk.month)
    engajamento['mes'][ind] = data_wrk.month
    engajamento['ano'][ind] = data_wrk.year
print(engajamento)

           Data  Segundos  data_corte  mes   ano  mes_ano
0    03/08/2023        24  2023-08-03    8  2023    20238
1    04/08/2023         6  2023-08-04    8  2023    20238
2    05/08/2023         5  2023-08-05    8  2023    20238
3    06/08/2023         6  2023-08-06    8  2023    20238
4    07/08/2023         8  2023-08-07    8  2023    20238
..          ...       ...         ...  ...   ...      ...
251  10/04/2024        27  2024-04-10    4  2024    20244
252  11/04/2024        14  2024-04-11    4  2024    20244
253  12/04/2024        29  2024-04-12    4  2024    20244
254  13/04/2024        17  2024-04-13    4  2024    20244
255  14/04/2024        17  2024-04-14    4  2024    20244

[256 rows x 6 columns]


## Cálculo dos valores mensais

In [6]:
media_mes = engajamento.groupby(['mes_ano', 'mes', 'ano'], as_index=False).sum()
count_mes = engajamento.groupby(['mes_ano', 'mes', 'ano'], as_index=False).count()
print(media_mes)
print(count_mes)

   mes_ano  mes   ano  Segundos
0    20238    8  2023       200
1    20239    9  2023       190
2    20241    1  2024       220
3    20242    2  2024       916
4    20243    3  2024       682
5    20244    4  2024       373
6   202310   10  2023       188
7   202311   11  2023       213
8   202312   12  2023       200
   mes_ano  mes   ano  Data  Segundos  data_corte
0    20238    8  2023    29        29          29
1    20239    9  2023    30        30          30
2    20241    1  2024    31        31          31
3    20242    2  2024    29        29          29
4    20243    3  2024    31        31          31
5    20244    4  2024    14        14          14
6   202310   10  2023    31        31          31
7   202311   11  2023    30        30          30
8   202312   12  2023    31        31          31


In [7]:
count_mes = count_mes.sort_values(by=['ano', 'mes'], ascending=True)
media_mes = media_mes.sort_values(by=['ano', 'mes'], ascending=True)

In [8]:
media_mes['media'] = 0.00
media_mes['qtd'] = 0
for ind in range(0, len(media_mes)):
    media = round((int(media_mes['Segundos'][ind]) / int(count_mes['Data'][ind])),2)
    media_mes['media'][ind] = media
    media_mes['mes_ano'][ind] = str(media_mes['mes'][ind]) + '/' + str(media_mes['ano'][ind])
    media_mes['qtd'][ind] = count_mes['Data'][ind]
print(media_mes)

   mes_ano  mes   ano  Segundos  media  qtd
0   8/2023    8  2023       200   6.90   29
1   9/2023    9  2023       190   6.33   30
6  10/2023   10  2023       188   6.06   31
7  11/2023   11  2023       213   7.10   30
8  12/2023   12  2023       200   6.45   31
2   1/2024    1  2024       220   7.10   31
3   2/2024    2  2024       916  31.59   29
4   3/2024    3  2024       682  22.00   31
5   4/2024    4  2024       373  26.64   14


# Relatório mensal de tempo médio de engajamento

In [9]:
fig = px.line(media_mes, x="mes_ano", y="media", title= ' Engajamento médio mensal de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Engajamento_mensal_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

# Cálculo do percentual de aumento do tempo médio

In [10]:
soma_antes = 0
qtd_antes = 0
soma_apos = 0
qtd_apos = 0
for ind in range(0, len(media_mes)):
    if media_mes['ano'][ind] == 2024:
        if media_mes['mes'][ind] == 1:
            soma_antes = soma_antes + int(media_mes['Segundos'][ind])
            qtd_antes = qtd_antes + int(media_mes['qtd'][ind])
        else:    
            soma_apos = soma_apos + int(media_mes['Segundos'][ind])
            qtd_apos = qtd_apos + int(media_mes['qtd'][ind])
    else:
        soma_antes = soma_antes + int(media_mes['Segundos'][ind])
        qtd_antes = qtd_antes + int(media_mes['qtd'][ind])
soma_antes = round((soma_antes / qtd_antes),2)
soma_apos = round((soma_apos / qtd_apos),2)
print(soma_antes)
print(soma_apos)

6.65
26.64


In [11]:
aumento = round((((soma_apos - soma_antes) / soma_antes) * 100),2)
print(aumento)

300.6
